# Building Dataset

In [194]:
import torch

alphabet = {chr(i): i - 97 for i in range(97, 123)}
alphabet["S"] = 26; alphabet["E"] = 27
rev_alphabet = {v: k for k, v in alphabet.items()}

def get_bigram_counts_with_delay(delay: int, names: list[torch.Tensor]):
    counts = torch.ones(len(alphabet), len(alphabet), dtype=torch.float) # .ones for smoothing
    for name in names:
        for bigram in zip(name, name[delay:]):
            counts[bigram[0], bigram[1]] += 1
    counts = counts / counts.sum(dim=1, keepdim=True)
    return counts

def build_dataset(max_delay: int):
    names = ["S" * max_delay + name.strip().lower() + "E" for name in open("data/names.txt").readlines()]
    names = list(filter(lambda x: "-" not in x and " " not in x, names))

    # Build dataset
    names = ["S" + name.strip().lower() + "E" for name in open("data/names.txt", "r").readlines()]
    names = list(filter(lambda x: "-" not in x, names))

    # Convert names to tensors
    names = [torch.tensor([alphabet[char] for char in name]) for name in names]
    
    # Create the counts matrix for each delay
    counts_per_delay = [get_bigram_counts_with_delay(delay, names) for delay in range(1, max_delay + 1)]

    return names, counts_per_delay

def display_name(name: list[int]):
    return "".join([rev_alphabet[i] for i in name]).replace("S", "").replace("E", "")

bigram_names, bigram_counts = build_dataset(1)
delay_names, delay_counts = build_dataset(3)

## Basic Models: Bigram, Delay, NN

In [195]:
# A series of models that give a probability distribution over the next character
from functools import reduce
from torch import nn

class BigramModel(nn.Module):
    def __init__(self, counts):
        super().__init__()
        self.counts = counts

    def forward(self, name: list[int]):
        probs = self.counts[0][name[-1], :]
        return nn.functional.normalize(probs, p=1.0, dim=0)

class DelayModel(nn.Module):
    def __init__(self, counts):
        super().__init__()
        self.counts = counts
    
    def forward(self, name: list[int]):
        rows = [
            self.counts[i][name[-(i + 1)], :]
            for i in range(len(self.counts))
        ]

        probs = reduce(lambda a, b: a * b, rows)
        return nn.functional.normalize(probs, p=1.0, dim=0)

class NNDelayModel(nn.Module):
    def __init__(self, counts):
        super().__init__()
        self.counts = counts
    
    def forward(self, name: list[int]):
        rows = [
            self.counts[i][name[-(i + 1)], :]
            for i in range(len(self.counts))
        ]

        probs = reduce(lambda a, b: a * b, rows)
        return nn.functional.normalize(probs, p=1.0, dim=0)


### Helper Functions

In [206]:
# Given a model, do some testing of it
def generate_name(model: torch.nn.Module, initial: list[int], parse_output=lambda x: x, prep_input=lambda x: x):
    name = [*initial] # Make a copy, otherwise you're updating the actual object, which is passed by pointer
    i = 0
    probs = ''
    while name[-1] != alphabet["E"] and i < 25:
        probs = parse_output(model(prep_input(name)))
        next_letter_idx = torch.multinomial(probs, 1).item()
        name.append(next_letter_idx)
        i += 1

    return name[1:-1]

def name_nll(model: torch.nn.Module, name: list[int], s_buffer: list[int], parse_output=lambda x: x, prep_input=lambda x: x):
    if isinstance(name, torch.Tensor):
        name = name.tolist()
    nll = 0
    full_name = s_buffer + name
    s_buffer_len = len(full_name) - len(name)
    
    for i in range(len(name)):
        probs = parse_output(model(prep_input(full_name[:s_buffer_len + i])))

        if not torch.sum(probs).isclose(torch.ones(())):
            raise Exception("ERROR probs don't add to one: " + str(torch.sum(probs).item()), probs)

        nll += -torch.log(probs[name[i]])

    return nll.item()

def test_model(model, prefix=[alphabet["S"]], iters=100, parse_output=lambda x: x, prep_input=lambda x: x):
    for _ in range(iters):
        name = generate_name(model, prefix, parse_output=parse_output, prep_input=prep_input)
        print(display_name(name))
    
    total = 0
    count = 0
    for name in bigram_names:
        nll = name_nll(model, name, prefix, parse_output=parse_output, prep_input=prep_input)
        total += nll
        count += 1

    print("Average NLL: ", total / count)

### Testing

In [197]:
bigram_model = BigramModel(bigram_counts)
test_model(bigram_model, iters=100)

had
mot
tas

odazinastio
c
mmissarthuilq
madar
ss
ht
wrushereorinar
juarelbe
lustulvpirsamacle
momverinertharizawarof
laloran
dy
an
peryars
inatiernnolle
m
budreranelle
feeer
beriuamalbre
jus
madrlerooshrtt
heymamaimee
schaniusit
foshabwi
binnondg
heshavad
coudier
mmsarastoudort
gond
rqurstzery
llmagierdricle
hewililarl
wicylin
b
guere
rie
r
etes
s
vs
rosh
sal
wclfrarienntcqx
rckinanistorin
itrimitan
gtcy
budo
txico
gy
joscan
iabdrvierd
tetthiavorlvexm
cap
shaud
garxby
ty
rariesban
stuwr
wnen
mobosordexerckericarins
har
giatebrbal
st
obrhan
stb
bbantot
s
oh
ffefl
hokit
llien
gndery
ghrken
s
aulliexindathanestharilm
ham
josepae
e
oxewariede
edonobbr
sh
mond
c
murba
rl
qulard
loamoncl
ln
fel
bydus
pqan
wishnd
ory
ahthanen
michen
balbrd
Average NLL:  25.06748729982841


In [198]:
delay_model = DelayModel(delay_counts)
test_model(delay_model, prefix=3*[alphabet["S"]])

rie
rrellie
iera
murie
eie
aser
rem
ar
rando
arr
err
alen
rda
ele
ete
erig
ari
lar
ari
arie
ell
rar
erie
ann
ier
aad
are
are
erie
ale
har
are
ron
arr
are
rrn
ale
har
rre
rin
er
arnn
ran
rial
ell
ran
arrin
ral
arae
dar
are
ire
are
lar
eor
rir
maer
las
err
aue
arn
rac
eren
allan
rea
aerie
eer
are
ale
ayle
ardie
lare
ole
erie
ara
rry
ris
rro
rin
rel
ran
lar
and
ell
arden
are
arr
ter
ene
ran
aro
arro
sanee
adle
onn
ron
arrie
ree
are
rin
Average NLL:  40.08920456905476


## Recurrent Neural Network - scratch

In [199]:
# RNN
def one_hot_encode_letter(letter: int):
    one_hot = torch.zeros(len(alphabet))
    one_hot[letter] = 1
    return one_hot
    
class RNN(nn.Module):
    def __init__(self, param_size: int):
        super().__init__()
        self.U = nn.Parameter(torch.randn(param_size, param_size))
        self.W = nn.Parameter(torch.randn(param_size, param_size))
        self.V = nn.Parameter(torch.randn(param_size, param_size))
        self.h = torch.randn(param_size)
        self.bias_h = nn.Parameter(torch.randn(param_size))
        self.bias_o = nn.Parameter(torch.randn(param_size))
        self.softmax = nn.Softmax(dim=0)

    # x should be a one-hot vector
    def forward(self, x):
        x = one_hot_encode_letter(x)
        self.h = torch.tanh(self.U @ x + self.W @ self.h + self.bias_h)
        return self.softmax(self.bias_o + self.V @ self.h)

    def detach_h(self):
        self.h = self.h.detach()

rnn_model = RNN(len(alphabet))
test_model(rnn_model)

ixdgxsetsyssxqxqskeizzei
pgclipxgylqxdszsieminsxq
xsyxmggxjxggszvmgomxggdx
xgyxibdxzygimixixggmmngg
iamxpixlgpdxxgxmgpsxpkxi
gcdsksgasgomgcmxaaglxilx
dlxaxpgemxxsxasxsyxqskex
cmgpsxaslxyxgesgoiqgzqkx
ggbdsgmssxgmsclxxxxxlxxk
ixgylxzzggysxixnbqxemged
xpmxpwgdsxwxgsgssxqsxsss
egomscsgssqscsqxesqqsqds
aqegcjsixsgyqzzzzgzziqqz
ggl
aixyxpxeicyiqipqiqrgzsio
xmqxdgxgsagnxixkgxclxxgx
gxsxaszqeksezggmzeagegim
cmxaixdgxixpglslywxgplix
yggydbebzeemgimsxnxgimxb
ixxgyxdsygysxsqqsszqegim
wsqxyxygezzzezeaezeizeiz
ec
phxsixygyqxqqzqqgqqqqqqs
msjxsxyxygygygxgxxgnnxxk
gmqscssgysekvgigcxigcgid
dc
saxmgxmxiilygxgpdmkxgxgx
cqspsqqqyslqqqqqqgolmxkx
geyyqczscceezzigggqqsngg
ggpsxiqxgygqzqgqysgmqsks
zogmsxsgxsxsggmxqxgskggx
mgbxslxqydewezzziegiescg
psclixyyliyxxuibxiwzzzzf
itipdsxisxiqxilszleemgem
pxcxyxwglyzeqzqpzqzzzqqz
yiqxixqgcnmxgmxcxgpdzpip
cnosbuckxxgkgxxkggggggcp
glspssxxgyxcglgilxygqqss
qgcskegggnsclgxgxkixygpd
nixxxmpkxxgxgggkxdgxyxgg
ditixbspipikxggmiskgqssg
mgpyxwgeyzegggiegggdxgps
bsepszsxyzwgzzz

In [200]:
# As expected, the untrained RNN outputs trash. Let's train.
for layer in rnn_model.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

opt = torch.optim.SGD(rnn_model.parameters(), lr=0.01)
for name in bigram_names:
    rnn_model.zero_grad()
    rnn_model.detach_h()
    loss = 0
    for bigram in zip(name, name[1:]):
        probs = rnn_model.forward(bigram[0])
        loss += -torch.log(probs[bigram[1]])
    loss.backward()
    opt.step()

test_model(rnn_model)

o
eahne
whxe
ephrruzksoekggggogegegii
onll
rxp
uztgokoggcigcilkikgegegg
gclgcllxdxgirxxysjpseqew
qgyiyxgddzaildlnilldwlgd
lgyxaykoalailgrlglgelied
lienri
hoial
aukehyhogsezekelgelkegeg
oalxiltxpilgldlgllllmlxi
ccdlolggxaileoxgxxxareh
hkokpiea
aourze
ua
hoaaoulecjresapweiegeseq
oqxdaehdhkrlleoea
oaoa
hwtvkeazeeeiyicgkzoiqiee
onlnylphxkxlielwxggedeeg
gilxnldxlgnxlxlileilkgee
gkxklaeyxxieaoegelgldeeg
g
ilxhpdxpxilxwegxszqqxegq
cgcysxysxomxndxgdklgekil
lkxilleixrenen
it
e
aaevreeoegiesieee
aihztasekzqgqeeeeaoeiale
gglxhlqxdskoklloalealaxi
ggcxilgonlldxlllipxwetkx
lxdxx
aewjroh
e
lwhapezzqqqzpsqzzeiqsege
gpdcasxyslollipxgxdxwlke
cnno
royayaw
adr
iaaeran
ochawajoehjzsaweahfdxtxd
x
m
ayawawu
hh
ru
o
aza

eeheoiatwuhcrurszjzskiks
oglsexyxeitxpgyszoyxqqzq
oidcxllttpilxxlitlxflpix
jyhfre
aexo

nzhzalaaydetuzxzekoexegg
gtxliolgyxpgyyxildxldxxg
xe
eewfrhjhezufsiftsqzsqbse
xggcnlurxhelkekzkokgkezl
lorextllkeiieioisipeklwe
rmiaarbe
wyyeuhrhyipzzzkgzkmesgeg
oilxixgxmxglemhrhxethpix
ggysy
egodstisxogq

## PyTorch Builtin RNN for reference

In [201]:
# Looks like builtin RNN does the same, maybe just not useful for bigram for some reason.
# Try utilizing full sequence, first with builtin RNN
# Restarting here, not going to use much of the above code. Want to try again from scratch.
class RNNv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(len(alphabet), len(alphabet))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, X):
        # X is matrix of size (seq_len, len(alphabet))
        output, _ = self.rnn(X, torch.randn(1, len(alphabet)))

        return self.softmax(output)

def name_to_matrix(name: list[int]) -> list[list[int]]:
    return torch.stack([one_hot_encode_letter(token) for token in name])

name_matrices = [name_to_matrix(name) for name in bigram_names]
rnn_model = RNNv2()

# Maybe problem from above was you weren't batching. At minimum should do multiple epochs through entire dataset with just one loss total
opt = torch.optim.Adam(rnn_model.parameters(), lr=0.1)

for epoch in range(10):
    loss = 0
    count = 0
    rnn_model.zero_grad()
    for name_matrix in name_matrices:
        # Predict a random letter in the name
        output = rnn_model(name_matrix)
        
        for i in range(output.shape[0]):
            loss += -torch.log(output[i][torch.nonzero(name_matrix[i])])
            count += 1
    
    loss /= count    
    loss.backward()
    opt.step()
    print("Epoch: ", epoch, " Loss: ", loss.item())


Epoch:  0  Loss:  3.332918405532837
Epoch:  1  Loss:  3.0117368698120117
Epoch:  2  Loss:  2.8645331859588623
Epoch:  3  Loss:  2.842525005340576
Epoch:  4  Loss:  2.7010793685913086
Epoch:  5  Loss:  2.612403631210327
Epoch:  6  Loss:  2.50447940826416
Epoch:  7  Loss:  2.4084012508392334
Epoch:  8  Loss:  2.3340466022491455
Epoch:  9  Loss:  2.2692861557006836


In [207]:
test_model(rnn_model, parse_output=lambda x: x[-1], prep_input=lambda x: name_to_matrix(x))

m
vp
eccveetg
dvkf
o
sg
rgrrooxmvlyywdozivnh
aa

p
viiipe
isnzpynw
w
npa
f
ksora
d
agj

bennnllfd
i
lvlhbylt
re
oiicariy
z
abengyyy
lbgvgh
qkmyaxj
ssjtt
hxsnwhq
ieayyxcqyc
olqewihtt

fyffkxihp
urgy

ccll
ukzttlmeuktbokuga
jh

mghvxld
rv
e
rurwpbgennmtxllol
louhds
rp
uoflkaswx
bjlym
zpnhmme
wiltcrdsttomvexltvbdd

g
wocwow
kewb
zzzcypumyrdctpqi
y
pa
shozpdetlj
llflkemdcbaagqr
ggql
lp
pmeaqbyeqbkrmaxiodxs
anyw
sdg
lawrhyyhyhnhk
fa
uvhawzqo
pemvhirpe
n
fgyfvsnipj
s
ta
pem
cj
cnxllu

rvbyvxdrwpnttgs
kku
j
rrtidkgblw
pcyaltyycmp
hc
t
z
sgergz

toogsqnhelioruefbrydzk
gjpnxs
na
jnuiiprzxiellla
eliessi
iqereyhhh

etdqpl
mcjceldd
bmawdke
f
pe
jthag
i
Average NLL:  23.968230487460616


In [215]:
# Now begins ATTENTION MECHANISM

class SingleHeadAttention(nn.Module):
    def __init__(self, embed_size: int, seq_len: int, v_size: int, kq_size: int):
        super().__init__()
        # Note that you can shrink the dimension of the query, key, and value vectors
        self.W_k = nn.Parameter(torch.randn(kq_size, embed_size))
        self.W_q = nn.Parameter(torch.randn(kq_size, embed_size))
        self.W_v = nn.Parameter(torch.randn(v_size, embed_size))
        self.d_k = torch.tensor(embed_size)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, X):
        # X represents a sequence. Each column is the embedding for a token.
        # X is (m x s), where m is the embedding size, and s is the sequence length
        # K, Q are the same size (m x s). V corresponds to output (n x s)

        K = self.W_k @ X # kq_size x s
        Q = self.W_q @ X # kq_size x s
        V = self.W_v @ X # v_size x s
        # You're not adding bias here, but that might be fine, because you want same mean of distribution?

        dot = K.T @ Q # (s x kq)(kq x s) s x s to represent attention between each pair
        dot /= torch.sqrt(self.d_k) # Because dot product of vectors with components distributed as N(0, 1) is distributed as N(0, d), and we want N(0, 1)

        # We softmax over the rows, to get an s x 1 column vector. Then each column of V is multiplied by the corresponding entry.
        # Is this broadcasted right?
        # To get really good at this, want to learn a) broadcasting rules and b) figure out symmetries of matrix multiplication
        # print("K: ", K.shape)
        # print("Q: ", Q.shape)
        # print("V: ", V.shape)
        # print("dot: ", dot.shape)

        z = V @ torch.softmax(dot, dim=1) # I don't think this should be normal matrix multiplication

        return self.softmax(z) # v_size x s

attention_model = SingleHeadAttention(28, 10, 28, 16)
test_model(attention_model, parse_output=lambda x: x[:, -1], prep_input=lambda x: name_to_matrix(x).T)

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size: int, seq_len: int, num_heads: int, v_size: int, kq_size: int):
        super().__init__()
        self.heads = nn.ModuleList([SingleHeadAttention(embed_size, seq_len, v_size, kq_size) for _ in range(num_heads)])

        # Head outputs (v_size x s) are concatenated into a (v_size * num_heads x s) matrix. And want (v_size x s) output, so multiply by (v_size x v_size * num_heads)
        self.W = nn.Parameter(torch.randn(v_size, v_size * num_heads))
        self.d_k = torch.tensor(embed_size)

    def forward(self, X):
        z = torch.cat([head(X) for head in self.heads], dim=0) # (v_size * num_heads x s)
        output = self.W @ z

        return output

class Transformer(nn.Module):
    def __init__(self, seq_len: int, embed_size: int, output_size: int):
        super().__init__()
    
    def forward(self, X):
        # Positional encoding

        # Attention
        # Calculate K, V, Q by separate linear layers from X

        # Batch normalization

        # Feedforward

        pass

ydrpyhx
hwezzdlrkwcchbqmvdcqieq
dp
divkkvzilzglrhoopopeekj
wienxfjczxpfc
vvad
qa
sgvwxwazxdrewuacilkxiqo
vppwadtymulxnfcknydnwfhg
nfyxiesilkcubbqzhhiboxfq
xxuu
y
pngmfcumvashatxfkgxjcfcl
x
xcwhgaxa
xi
xilxdkbrgzqitpvcoviinvh
wacslxjriescmg
wabxreequcrtpmpzbyzdzuxj
dwikrpykxiwzjeroizxozpsd
ywdbquogdubkbfcshgxgraf
xciwhrzflwui
xclhbhyuxh
xaoizqcguoiltthjdrltfulg
uahbxgxo
zvzmweelfliyuaymweuiszxm
wyvhsliilxhqwvwdmcaofn
gpnyxzlicimmblwnzbhqopht
bvfwpo

fabgywdxxfa
cdhfgrugfzdtkezhjeqlxagd
yxrqlwuwciluonvyeasq
wxbfnzedxxrkwovc
eququarv
ilzncxjsyhuj
hyngvphvjlmpvbyuznili
zikxwvfou
xizgdfejxlqffdxineanoqu
xalxpychqalcaxvadujreu
kjcdxafrfnuwffai
xjgdrqrakij
xmakxxfizlyloasgvsdoqqn
qwyyagpjmsxbhntzbefqghnt
xadm
auxpdqimoirmvgkpwnop
lh
xxv
fgaxfgzv
xcuzzxgieamjmiqzkohtpdp
yywmpziwwbyp
wewughbreddflomlyetvdkkn

xljdwmelmouohlsfyktmingc
wugtauuyx
xxligibrgkjscgj
ximfzixwohlrs
zwalbmkic
xdvriufwfr
dgwvpfzbzwpmnapykorfgnnh
vx
bxlxoqckypxrervlyqzavm
xwd
yykxgkxxnznvpxom
yxfggabyahiubcqmuxmfu
wwvukoui